In [ ]:
# -*- coding: utf-8 -*-
# ArcMap 10.x (Python 2.7) - Unir KMZ/KML (solo Puntos) en un único Feature Class
# Autor: ChatGPT

import arcpy, os, sys, glob, shutil
arcpy.env.overwriteOutput = True

# ========= RUTAS (ajustadas a tu caso) =========
IN_FOLDER   = r"D:\semillero"                 # Carpeta con KMZ/KML
WORK_FOLDER = r"D:\semillero\_tmpKML"         # Carpeta temporal (se crea si no existe)
OUT_GDB     = r"D:\semillero\salida.gdb"      # GDB de salida
OUT_POINTS  = "KMZ_Puntos_Unidos"             # Nombre del FC final
# ==============================================

def ensure_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)

def ensure_gdb(gdb_path):
    if not arcpy.Exists(gdb_path):
        folder, name = os.path.split(gdb_path)
        ensure_folder(folder)
        arcpy.CreateFileGDB_management(folder, name)

def kml_to_layer(in_kml, out_folder, out_name):
    arcpy.AddMessage(u"Convirtiendo: {0}".format(os.path.basename(in_kml)))
    # Crea <out_name>.gdb y <out_name>.lyr en out_folder
    arcpy.KMLToLayer_conversion(in_kml, out_folder, out_name, "NO_GROUNDOVERLAY")
    return os.path.join(out_folder, out_name + ".gdb")

def find_points_fc(placemarks_path):
    """Devuelve la ruta a la FC de puntos dentro de 'Placemarks' (si existe)."""
    if not arcpy.Exists(placemarks_path):
        return None
    arcpy.env.workspace = placemarks_path
    fcs = arcpy.ListFeatureClasses()
    if not fcs:
        return None
    # Buscar por nombre estándar o por tipo de geometría
    for fc in fcs:
        name = os.path.basename(fc)
        if name.lower() == "points":
            return os.path.join(placemarks_path, fc)
    # Si el nombre es distinto, detectamos por geometryType
    for fc in fcs:
        desc = arcpy.Describe(fc)
        if getattr(desc, "shapeType", "").lower() == "point":
            return os.path.join(placemarks_path, fc)
    return None

def main():
    try:
        # Preparación
        ensure_folder(IN_FOLDER)
        ensure_folder(WORK_FOLDER)
        ensure_gdb(OUT_GDB)

        # Recolectar KML/KMZ
        files = glob.glob(os.path.join(IN_FOLDER, "*.kmz")) + \
                glob.glob(os.path.join(IN_FOLDER, "*.kml"))
        if not files:
            arcpy.AddWarning(u"No se encontraron .kmz/.kml en {0}".format(IN_FOLDER))
            return

        puntos_inputs = []

        # Convertir y recolectar la capa de puntos de cada archivo
        for in_file in files:
            base = os.path.splitext(os.path.basename(in_file))[0]
            out_name = base[:40].replace(" ", "_")
            gdb_created = kml_to_layer(in_file, WORK_FOLDER, out_name)

            placemarks = os.path.join(gdb_created, "Placemarks")
            points_fc = find_points_fc(placemarks)
            if points_fc:
                puntos_inputs.append(points_fc)
            else:
                arcpy.AddWarning(u"Sin puntos en: {0}".format(os.path.basename(in_file)))

        if not puntos_inputs:
            arcpy.AddWarning(u"No se hallaron puntos en los KML/KMZ procesados.")
            return

        # Fusionar (o copiar si solo hay uno)
        out_fc = os.path.join(OUT_GDB, OUT_POINTS)
        if len(puntos_inputs) == 1:
            arcpy.AddMessage(u"Solo un origen con puntos. Copiando a salida...")
            arcpy.CopyFeatures_management(puntos_inputs[0], out_fc)
        else:
            arcpy.AddMessage(u"Fusionando {0} capas de puntos...".format(len(puntos_inputs)))
            arcpy.Merge_management(puntos_inputs, out_fc)

        arcpy.AddMessage(u"✔ Listo. Salida: {0}".format(out_fc))
        arcpy.AddMessage(u"CRS original: WGS 1984 (KML). Proyecta si lo necesitas.")

        # (Opcional) Limpieza del temporal:
        # Descomenta si deseas borrar la carpeta temporal al final
        # try:
        #     shutil.rmtree(WORK_FOLDER)
        #     arcpy.AddMessage(u"Temporal eliminado: {0}".format(WORK_FOLDER))
        # except Exception as _:
        #     arcpy.AddWarning(u"No se pudo borrar el temporal. Ciérralo si está abierto e inténtalo luego.")

    except Exception as e:
        sys.stderr.write("Error: {0}\n".format(e))
        sys.stderr.write(arcpy.GetMessages(2))

if __name__ == "__main__":
    main()
